Plan-and-Execute with LastMile Auto Instrumentation

This notebook demonstrates the concept of a "Plan-and-Execute" LangChain agent with LastMile auto instrumentation. The agent accomplishes an objective by first planning what to do and then executing the sub-tasks. LastMile auto instrumentation is used to trace and monitor the execution of the agent.

In [1]:
!pip install langchain
!pip install langchain_community
!pip install langchain_core
!pip install langchain_experimental
!pip install langchain_openai
!pip install wikipedia
!pip install numexpr

!pip install "tracing_auto_instrumentation[langchain]"

  Using cached PyYAML-6.0.1-cp310-cp310-macosx_11_0_arm64.whl.metadata (2.1 kB)
  Using cached SQLAlchemy-2.0.30-cp310-cp310-macosx_11_0_arm64.whl.metadata (9.6 kB)
  Using cached aiohttp-3.9.5-cp310-cp310-macosx_11_0_arm64.whl.metadata (7.5 kB)
  Using cached async_timeout-4.0.3-py3-none-any.whl.metadata (4.2 kB)
  Using cached numpy-1.26.4-cp310-cp310-macosx_11_0_arm64.whl.metadata (61 kB)
  Using cached pydantic-2.7.1-py3-none-any.whl.metadata (107 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl.metadata (4.0 kB)
  Using cached attrs-23.2.0-py3-none-any.whl.metadata (9.5 kB)
  Using cached frozenlist-1.4.1-cp310-cp310-macosx_11_0_arm64.whl.metadata (12 kB)
  Using cached multidict-6.0.5-cp310-cp310-macosx_11_0_arm64.whl.metadata (4.2 kB)
  Using cached yarl-1.9.4-cp310-cp310-macosx_11_0_arm64.whl.metadata (31 kB)
  Using cached marshmallow-3.21.2-py3-none-any.whl.metadata (7.1 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached jsonpatch-1.3

In [8]:
# Imports

from langchain.chains import LLMMathChain
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_core.tools import Tool
from langchain_experimental.plan_and_execute import (
    PlanAndExecute,
    load_agent_executor,
    load_chat_planner,
)
from langchain_openai import ChatOpenAI, OpenAI

# LastMile Auto Instrumentation Setup

An instance of `LangChainInstrumentor` is created with a project name. The `instrument()` method is called to instrument the code for tracing and monitoring.

In [10]:
import dotenv

dotenv.load_dotenv()

True

In [6]:
from tracing_auto_instrumentation.langchain import LangChainInstrumentor

# Create an instance of LangChainInstrumentor and instrument the code
instrumentor = LangChainInstrumentor(project_name="Plan-and-Execute Example")
instrumentor.instrument()

2024-05-21 20:55:19,331 - Starting new HTTPS connection (1): lastmileai.dev:443
2024-05-21 20:55:19,477 - https://lastmileai.dev:443 "GET /api/evaluation_projects/list?name=Plan-and-Execute+Example HTTP/1.1" 200 41
2024-05-21 20:55:19,479 - Starting new HTTPS connection (1): lastmileai.dev:443
2024-05-21 20:55:19,564 - https://lastmileai.dev:443 "POST /api/evaluation_projects/create HTTP/1.1" 200 261


# Tools

The tools that the agent will use for executing sub-tasks are defined. In this example, we have two tools:
- "Wikipedia" tool: Uses the `WikipediaAPIWrapper` to look up information on Wikipedia.
- "Calculator" tool: Uses the `LLMMathChain` to perform mathematical calculations.

In [11]:
# Define the tools that the agent will use for executing sub-tasks

wikipedia = WikipediaAPIWrapper()
llm = OpenAI(temperature=0)
llm_math_chain = LLMMathChain.from_llm(llm=llm, verbose=True)
tools = [
    Tool(
        name="Wikipedia",
        func=wikipedia.run,
        description="useful for when you need to look up information on Wikipedia",
    ),
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to perform mathematical calculations",
    ),
]


2024-05-21 20:57:37,896 - load_ssl_context verify=True cert=None trust_env=True http2=False
2024-05-21 20:57:37,897 - load_verify_locations cafile='/opt/homebrew/Caskroom/miniconda/base/envs/tai1/lib/python3.10/site-packages/certifi/cacert.pem'
2024-05-21 20:57:37,921 - load_ssl_context verify=True cert=None trust_env=True http2=False
2024-05-21 20:57:37,922 - load_verify_locations cafile='/opt/homebrew/Caskroom/miniconda/base/envs/tai1/lib/python3.10/site-packages/certifi/cacert.pem'


# Run Example

The agent is run with a sample question that involves looking up the population of the capital city of the country where the Eiffel Tower is located and calculating the square root of that number. The result is printed.

In [12]:
model = ChatOpenAI(temperature=0)
planner = load_chat_planner(model)
executor = load_agent_executor(model, tools, verbose=True)
agent = PlanAndExecute(planner=planner, executor=executor)

# ## Run Example

# Run the agent with a sample question
question = "What is the population of the capital city of the country where the Eiffel Tower is located? Calculate the square root of that number."
result = agent.run(question)

2024-05-21 20:57:40,539 - load_ssl_context verify=True cert=None trust_env=True http2=False
2024-05-21 20:57:40,541 - load_verify_locations cafile='/opt/homebrew/Caskroom/miniconda/base/envs/tai1/lib/python3.10/site-packages/certifi/cacert.pem'
2024-05-21 20:57:40,568 - load_ssl_context verify=True cert=None trust_env=True http2=False
2024-05-21 20:57:40,569 - load_verify_locations cafile='/opt/homebrew/Caskroom/miniconda/base/envs/tai1/lib/python3.10/site-packages/certifi/cacert.pem'
/opt/homebrew/Caskroom/miniconda/base/envs/tai1/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
2024-05-21 20:57:40,611 - Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'content': "Let's first understand the problem and devise a plan to solve the problem. Please output the 



> Entering new AgentExecutor chain...


2024-05-21 20:57:43,253 - receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Wed, 22 May 2024 00:57:43 GMT'), (b'Content-Type', b'application/json'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'openai-organization', b'lastmile-ai'), (b'openai-processing-ms', b'538'), (b'openai-version', b'2020-10-01'), (b'strict-transport-security', b'max-age=15724800; includeSubDomains'), (b'x-ratelimit-limit-requests', b'10000'), (b'x-ratelimit-limit-tokens', b'2000000'), (b'x-ratelimit-remaining-requests', b'9999'), (b'x-ratelimit-remaining-tokens', b'1999667'), (b'x-ratelimit-reset-requests', b'6ms'), (b'x-ratelimit-reset-tokens', b'9ms'), (b'x-request-id', b'req_093e024ef74d3596e6ae7e6ccd8b0fdc'), (b'CF-Cache-Status', b'DYNAMIC'), (b'Server', b'cloudflare'), (b'CF-RAY', b'8878da88cbd98cd7-EWR'), (b'Content-Encoding', b'gzip'), (b'alt-svc', b'h3=":443"; ma=86400')])
2024-05-21 20:57:43,254 - HTTP Request: POST https://api.openai.com/v1/c

Thought: The Eiffel Tower is located in France.

Action:
```
{
  "action": "Final Answer",
  "action_input": "France"
}
```

> Finished chain.


> Entering new AgentExecutor chain...


2024-05-21 20:57:45,711 - receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Wed, 22 May 2024 00:57:45 GMT'), (b'Content-Type', b'application/json'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'openai-organization', b'lastmile-ai'), (b'openai-processing-ms', b'780'), (b'openai-version', b'2020-10-01'), (b'strict-transport-security', b'max-age=15724800; includeSubDomains'), (b'x-ratelimit-limit-requests', b'10000'), (b'x-ratelimit-limit-tokens', b'2000000'), (b'x-ratelimit-remaining-requests', b'9999'), (b'x-ratelimit-remaining-tokens', b'1999644'), (b'x-ratelimit-reset-requests', b'6ms'), (b'x-ratelimit-reset-tokens', b'10ms'), (b'x-request-id', b'req_7b1cff78891fa58154f59a5fbc41eb39'), (b'CF-Cache-Status', b'DYNAMIC'), (b'Server', b'cloudflare'), (b'CF-RAY', b'8878da9649618cd7-EWR'), (b'Content-Encoding', b'gzip'), (b'alt-svc', b'h3=":443"; ma=86400')])
2024-05-21 20:57:45,713 - HTTP Request: POST https://api.openai.com/v1/

...

2024-05-21 20:57:55,918 - receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Wed, 22 May 2024 00:57:55 GMT'), (b'Content-Type', b'application/json'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'openai-organization', b'lastmile-ai'), (b'openai-processing-ms', b'1143'), (b'openai-version', b'2020-10-01'), (b'strict-transport-security', b'max-age=15724800; includeSubDomains'), (b'x-ratelimit-limit-requests', b'10000'), (b'x-ratelimit-limit-tokens', b'2000000'), (b'x-ratelimit-remaining-requests', b'9999'), (b'x-ratelimit-remaining-tokens', b'1999479'), (b'x-ratelimit-reset-requests', b'6ms'), (b'x-ratelimit-reset-tokens', b'15ms'), (b'x-request-id', b'req_cc2a9cd4d292422f674bb4d197d4d605'), (b'CF-Cache-Status', b'DYNAMIC'), (b'Server', b'cloudflare'), (b'CF-RAY', b'8878dad43a4a8cd7-EWR'), (b'Content-Encoding', b'gzip'), (b'alt-svc', b'h3=":443"; ma=86400')])
2024-05-21 20:57:55,920 - HTTP Request: POST https://api.openai.com/v1

I have calculated the square root of the population of Paris, which is approximately 2,102,650 residents, to be 1450.0517232154168.
Action:
```
{
  "action": "Final Answer",
  "action_input": "The square root of the population of Paris, approximately 2,102,650 residents, is 1450.0517232154168."
}
```

> Finished chain.


> Entering new AgentExecutor chain...


2024-05-21 20:57:57,385 - receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Wed, 22 May 2024 00:57:57 GMT'), (b'Content-Type', b'application/json'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'openai-organization', b'lastmile-ai'), (b'openai-processing-ms', b'1117'), (b'openai-version', b'2020-10-01'), (b'strict-transport-security', b'max-age=15724800; includeSubDomains'), (b'x-ratelimit-limit-requests', b'10000'), (b'x-ratelimit-limit-tokens', b'2000000'), (b'x-ratelimit-remaining-requests', b'9999'), (b'x-ratelimit-remaining-tokens', b'1999501'), (b'x-ratelimit-reset-requests', b'6ms'), (b'x-ratelimit-reset-tokens', b'14ms'), (b'x-request-id', b'req_bec895b1e72f1f2d7d154b8844b508c0'), (b'CF-Cache-Status', b'DYNAMIC'), (b'Server', b'cloudflare'), (b'CF-RAY', b'8878dadd3bf28cd7-EWR'), (b'Content-Encoding', b'gzip'), (b'alt-svc', b'h3=":443"; ma=86400')])
2024-05-21 20:57:57,386 - HTTP Request: POST https://api.openai.com/v1

Thought: The final answer is the square root of the population of Paris, which is approximately 1450.0517232154168.

Action:
```
{
  "action": "Final Answer",
  "action_input": "The square root of the population of Paris, approximately 2,102,650 residents, is approximately 1450.0517232154168."
}
```



> Finished chain.


2024-05-21 20:57:57,685 - https://lastmileai.dev:443 "POST /api/rag_events/create HTTP/1.1" 200 None
2024-05-21 20:57:57,687 - Starting new HTTPS connection (1): lastmileai.dev:443
2024-05-21 20:57:57,774 - https://lastmileai.dev:443 "POST /api/rag_events/create HTTP/1.1" 200 None
2024-05-21 20:57:57,777 - Starting new HTTPS connection (1): lastmileai.dev:443
2024-05-21 20:57:57,888 - https://lastmileai.dev:443 "POST /api/rag_events/create HTTP/1.1" 200 985
2024-05-21 20:57:57,890 - Starting new HTTPS connection (1): lastmileai.dev:443
2024-05-21 20:57:57,964 - https://lastmileai.dev:443 "POST /api/rag_events/create HTTP/1.1" 200 None
2024-05-21 20:57:57,966 - Starting new HTTPS connection (1): lastmileai.dev:443
2024-05-21 20:57:58,056 - https://lastmileai.dev:443 "POST /api/rag_events/create HTTP/1.1" 200 None
2024-05-21 20:57:58,059 - Starting new HTTPS connection (1): lastmileai.dev:443
2024-05-21 20:57:58,139 - https://lastmileai.dev:443 "POST /api/rag_events/create HTTP/1.1" 200 

In [13]:
print(result)

The square root of the population of Paris, approximately 2,102,650 residents, is approximately 1450.0517232154168.




## Imports

The necessary libraries are imported, including `langchain` for the agent and tools, `langchain_openai` for the language models, and `lastmile_eval` for auto instrumentation.




## Planner, Executor, and Agent

The planner, executor, and agent are set up using the `load_chat_planner`, `load_agent_executor`, and `PlanAndExecute` classes, respectively. The planner and executor use a `ChatOpenAI` model, and the executor is configured with the defined tools.
